In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def euler_step(f, xn, tn, delta_t):
    '''f is the right hand side of first order ODE x_dot = f 
       xn is the inital x value
       t is the timestep
       tn is initial t value'''
    xn = xn + f(xn, tn)*delta_t
    return xn

In [10]:
def solve_to(xn, tn, tf, delta_tmax,f, method):
    if method in ('euler', 'Euler', 'e', 'E'):
        print('euler')
        step_function = euler_step
    elif method in ('rk', 'RK', 'r', 'Runge-Kutta 4', 'RK4', 'runge-kutta4', 'rk4'):
        print('rk4')
        step_function = RK4_step
    while tn + delta_tmax < tf: 
        xn = step_function(f, xn, tn, delta_tmax)
        tn += delta_tmax
    else:
        diff = 1 - tn
        newdelta_tmax = diff
        tn += diff
        xn = step_function(f, xn, tf, newdelta_tmax)
        return xn
    

In [11]:
def solve_ode(f, delta_t, tf, method):
    estimation = solve_to(1,0,tf,delta_t,f, method)
    print(estimation)
    error = (np.exp(tf) - estimation)/np.exp(tf) *100
    return(error)

In [12]:
timesteps = np.arange(0.001,1,0.001)
errors = []
for timestep in timesteps:
    errors.append(solve_ode(lambda x,t: x, timestep, 1, 'euler'))

euler
2.7169239322358942
euler
2.715568520651726
euler
2.7142182936090817
euler
2.712865123051451
euler
2.711517122929372
euler
2.7101823480802474
euler
2.70883654156081
euler
2.707487833210312
euler
2.7061603699370935
euler
2.7048138294215263
euler
2.7034937940237547
euler
2.702192290752052
euler
2.7008369085836454
euler
2.69955876335486
euler
2.698237428119991
euler
2.6969346610621807
euler
2.695586201074523
euler
2.694319931288632
euler
2.6930107115536885
euler
2.691588029073604
euler
2.6904152960475773
euler
2.689129300270696
euler
2.6878399533568715
euler
2.686531457163568
euler
2.685063838389972
euler
2.6839858505309113
euler
2.6825046939175814
euler
2.6813850573098
euler
2.6801598496451136
euler
2.678858590739219
euler
2.6775514283709687
euler
2.676277532245269
euler
2.675044551506893
euler
2.673828690303299
euler
2.6725753962021845
euler
2.671200643739325
euler
2.66968794223062
euler
2.668773421604128
euler
2.6675499129233584
euler
2.665836331487419
euler
2.665085928251251
eule

2.041151
euler
2.040236
euler
2.039319
euler
2.0383999999999998
euler
2.037479
euler
2.036556
euler
2.035631
euler
2.034704
euler
2.033775
euler
2.032844
euler
2.031911
euler
2.030976
euler
2.030039
euler
2.0291
euler
2.028159
euler
2.027216
euler
2.026271
euler
2.025324
euler
2.024375
euler
2.023424
euler
2.022471
euler
2.021516
euler
2.020559
euler
2.0196
euler
2.018639
euler
2.017676
euler
2.016711
euler
2.015744
euler
2.0147749999999998
euler
2.013804
euler
2.0128310000000003
euler
2.011856
euler
2.010879
euler
2.0099
euler
2.008919
euler
2.007936
euler
2.006951
euler
2.005964
euler
2.004975
euler
2.003984
euler
2.0029909999999997
euler
2.001996
euler
2.000999


In [13]:
def RK4_step(f, xn, tn, delta_t): 
    k1 = delta_t*f(xn, tn) 
    k2 = delta_t*f(xn + 0.5 * k1 , tn + 0.5 * delta_t) 
    k3 = delta_t*f(xn + 0.5 * k2 , tn + 0.5 * delta_t) 
    k4 = delta_t*f(xn + k3, tn + delta_t) 
   
    xn = xn + (1.0/6.0)*(k1 + 2 * k2 + 2 * k3 + k4) 
    return (xn)

In [14]:
RK4_step(lambda x,t: x, 1, 0, 0.2)

1.2214

In [15]:
solve_ode(lambda x,t: x, 0.1, 1, 'euler')

euler
2.5937424601000005


4.581547323576972

In [16]:
solve_ode(lambda x,t: x, 0.1, 1, 'rk4')

rk4
2.7182797441351663


7.667799037627856e-05